Detailed ad retrieval from 3 websites ingatlanok.hu,ingatlan.com, otthoncentrum.

Step 1: retrive unique ad's urls from the search engine result to 'urls' variable for a give district
Step 2: use the urls to retrive the ad homepage <body> tag
Step 3: save the ads into source specific .csv

In [1]:
    #urls
    url1a = 'https://ingatlan.com/szukites/elado+lakas+xi-ker+48-55-m2+ar-szerint?page='
    url1b = 'https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page='
    url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
    url2a = 'https://ingatlanok.hu/elado/lakas+haz/budapest-xi-ker:48nm-tol;55nm-ig?record=0&num=100'
    url2b = 'https://ingatlanok.hu/elado/lakas+haz/budapest-vi-ker:38nm-tol;55nm-ig?record=0&num=100'
    url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    url3a = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest11-kerulet/hely-id:budapest11-kerulet,/netto-alapterulet:45~55/emelet:1,8'
    url3b = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest06-kerulet/hely-id:budapest06-kerulet,/netto-alapterulet:38~55/emelet:1,8'

    url1 = 'https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page='
    url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
    url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    


In [2]:
import requests
#from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
import urllib3
import warnings
warnings.filterwarnings('ignore')
import datetime as dtime
import time
import pandas as pd

In [3]:
#searchpage retrival

In [4]:
#ingatlan.com, multi-page
def _multi_get(url, n):
    '''
    gets multipages session based webpages like search results specialized to ingatlan.com site
    
    url for URL
    n for number of pages to follow
    
    returns ads specific url in list
    '''
    urls = []
    link = url + str(n)
    print(link)
    http = urllib3.PoolManager(headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    response = http.request('GET', link)
    soup = BeautifulSoup(response.data, 'lxml')
    l = soup.find_all('div', attrs = {'class': 'listing js-listing listing--cluster-parent js-cluster-parent'})
    l = l + (soup.find_all('div', attrs = {'class': 'listing js-listing'}))
    for i in l:
            urls.append('https://ingatlan.com'+i.find('a')['href'])
    
    return urls

def multipagesSearch(url , v=10):
    '''
    wrapper for _multi_get(url, n) to introduce timer between page download 
    url for URL,
    v for number of pages to return
    
    _multi_get:
    gets multipages session based webpages like search results specialized to ingatlan.com site
    
    returns ad specific url in list
    '''
    i=1
    while i <= v:
        text = _multi_get(url, n=i)
        
        if 'csvtext' in locals():
            csvtext = csvtext + text
        else: csvtext = text
            
        time.sleep(1)
        i += 1
    return csvtext

#ingatlanok.hu, oc.hu, one page
def simpleSearch(url):
    '''
    gets simple one page website from the URL
    
    returns ad specific url for .hu and oc.hu pages
    '''
    r=requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    
    page = BeautifulSoup(r.content, 'lxml')
    body = page.body
    url_list = []
    if 'ingatlanok.hu' in url:
        a = body.find_all('div', attrs={'class': 'result-row'})
        for i in a:
            url_list.append(i.find_all('a')[1]['href'])
    if 'oc.hu' in url:
        a = body.find_all('div', attrs={'class': 'estate-list-box'})
        for i in a:
            url_list.append('https://www.oc.hu'+i.find_all('a')[0]['href'])
    return url_list

In [5]:
#compose for searPage url retrieval
def searchPage(district):
    '''
    retrives the search page results ad's url into a [list] according to the given district.
    Options:
    District: 'V', 'VI','XI'
    '''
    if district == 'V':
        url1 = 'https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page='
        url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
        url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    elif district == 'VI':
        url1 = 'https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page='
        url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-vi-ker:38nm-tol;55nm-ig?record=0&num=100'
        url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest06-kerulet/hely-id:budapest06-kerulet,/netto-alapterulet:38~55/emelet:1,8'
    elif district == 'XI':
        url1 = 'https://ingatlan.com/szukites/elado+lakas+xi-ker+48-55-m2+ar-szerint?page='
        url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-xi-ker:48nm-tol;55nm-ig?record=0&num=100'
        url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest11-kerulet/hely-id:budapest11-kerulet,/netto-alapterulet:45~55/emelet:1,8'
    
    hu = simpleSearch(url2)
    print('Hu search: ',len(hu))
    
    com = multipagesSearch(url1)
    print('Com search: ',len(com))
    
    cen = simpleSearch(url3)
    print('Cen search: ',len(cen))
    
    alls = [*hu,*com,*cen]
    print('All urls: ', len(alls))
    
    alls =[i for i in alls if 'http' in i]
    return alls

In [6]:
# detailed webpage retrival

In [7]:
def _simple(url):
    '''
    Returns the page <body> tag from url retrieval
    '''
    page = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    body = BeautifulSoup(page.content, 'lxml').body

    return body

def details(urls, district):
    '''
    Go through the list of urls, retrieves the body tag. 
    Sorted by url, saves the body tag of those that has the keyword specific to the url source 
    
    district: 'V', 'VI', 'XI'
    '''
    
    adshu = []
    adscom = []
    adscen = []
    n=0
    for i in urls:
        print(n,'\t',i)
        if 'ingatlanok.hu' in i:
            try:
                back = _simple(i)               
                txt = back.get_text()

                if 'Azonosító:' in txt:             
                    adshu.append(back)
                    
            except: pass
            
        elif 'oc.hu' in i:
            try:
                back = _simple(i)               
                txt = back.get_text()
                
                if 'Regisztrációs szám:' in txt:             
                    adscen.append(back)
            except: pass
            
        else:
            try:
                back = _simple(i)               
                txt = back.get_text()
                
                if 'Parkolás' in txt or 'Ingatlan állapota' in txt:             
                    adscom.append(back)
            except: pass
        
        n+=1 # counter
    
    now = dtime.datetime.now()
    m = [str(now.month) if len(str(now.month)) == 2 else '0' + str(now.month)]
    d = [str(now.day) if len(str(now.day)) == 2 else '0' + str(now.day)]
    y = str(now.year)
    
    with open('adshu'+ district + '_' + d[0] + m[0] + y + '.csv', 'w') as f:
                f.write('%s' % adshu)
    with open('adscom'+ district + '_' + d[0] + m[0] + y + '.csv', 'w') as f:
                f.write('%s' % adscom)
    with open('adscen'+ district + '_' + d[0] + m[0] + y + '.csv', 'w') as f:
                f.write('%s' % adscen)
    return adshu, adscom,adscen

In [8]:
def checkDownload(date, d = ['V','VI','XI']):
    '''
    Check the No of saved ads.
    Search by date
    Search by district in list format -> ['XX']
    '''
    for n in d:
        for i in ['adshu'+n,'adscom'+n,'adscen'+n]:
            try:
                f = i+ '_' + date +'.csv'
                with open(f, 'r') as r:
                    l = r.read()
                ll = l.split('</body>,')
                print('Saved ads in {}: {}'.format(i,len(ll)))
            except: print('No such file ' + f)

In [9]:
ads = []
for i in ['V','VI', 'XI']:
    urls = searchPage(i)
    ads.append(details(urls,i))

Hu search:  14
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=1
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=2
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=3
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=4
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=5
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=6
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=7
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=8
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=9
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=10
Com search:  45
Cen search:  3
All urls:  62
0 	 https://ingatlanok.hu/elado/lakas/budapest_v_ker/22-millio-ft;27-negyzetmeter;2-szoba;tegla-epitesu/9608119
1 	 https://ingatlanok.hu/elado/lakas/budapest_v_ker/36-millio-ft;32-negyzetm

1 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/39-millio-ft;49-negyzetmeter;2-szoba;tegla-epitesu/9919291
2 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/35-millio-ft;43-negyzetmeter;1-szoba;tegla-epitesu/9931045
3 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/69-millio-ft;52-negyzetmeter;2-1-szoba;tegla-epitesu/8854530
4 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/39-millio-ft;49-negyzetmeter;2-szoba;tegla-epitesu/9919312
5 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/51-millio-ft;51-negyzetmeter;2-szoba;nincs-megadva-epitesu/9775882
6 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/49-millio-ft;47-negyzetmeter;2-szoba;tegla-epitesu/9725543
7 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/57-millio-ft;55-negyzetmeter;3-szoba;tegla-epitesu/9765151
8 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/39-millio-ft;50-negyzetmeter;2-szoba;nincs-megadva-epitesu/9837024
9 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/34-millio-ft;43-negyzetm

72 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/29-millio-ft;44-negyzetmeter;1-szoba;tegla-epitesu/9430999
73 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/42-millio-ft;44-negyzetmeter;1-1-szoba;tegla-epitesu/9908708
74 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/32-millio-ft;40-negyzetmeter;1-szoba;tegla-epitesu/9941922
75 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/47-millio-ft;55-negyzetmeter;2-szoba;tegla-epitesu/9967484
76 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/39-millio-ft;43-negyzetmeter;2-szoba;tegla-epitesu/9873216
77 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/43-millio-ft;53-negyzetmeter;2-szoba;tegla-epitesu/9827360
78 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/39-millio-ft;52-negyzetmeter;1-1-szoba;tegla-epitesu/9932209
79 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/46-millio-ft;41-negyzetmeter;1-szoba;tegla-epitesu/9862274
80 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/43-millio-ft;54-negyzetmeter;

148 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/31281052
149 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/31187164
150 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+podmaniczky+utca/31169131
151 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+podmaniczky+utca/31046989
152 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+csengery+utca+74/30700720
153 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+vorosmarty+utca/31195702
154 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+vorosmarty+utca/31028440
155 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+szondi+utca/31097062
156 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+szondi+utca/29836724
157 	 https://ingatlan.com/vi-ker/elado+lakas/t

227 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+eotvos+utca/31331695
228 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+csengery+utca/31278202
229 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+eotvos+utca/31235893
230 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+felso+erdosor/31163779
231 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+felso+erdosor/30862897
232 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+felso+erdosor/30848986
233 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/30596008
234 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+sziv+utca/31425787
235 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+sziv+utca/31149451
236 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-ep

315 	 https://www.oc.hu/ingatlanok/H401440?c=79cc83b7
316 	 https://www.oc.hu/ingatlanok/LK478425?c=79cc83b7
317 	 https://www.oc.hu/ingatlanok/LK485221?c=79cc83b7
318 	 https://www.oc.hu/ingatlanok/LK497224?c=79cc83b7
319 	 https://www.oc.hu/ingatlanok/LK493065?c=79cc83b7
320 	 https://www.oc.hu/ingatlanok/LK498557?c=79cc83b7
321 	 https://www.oc.hu/ingatlanok/LK499358?c=79cc83b7
322 	 https://www.oc.hu/ingatlanok/LK500476?c=79cc83b7
323 	 https://www.oc.hu/ingatlanok/LK491591?c=79cc83b7
324 	 https://www.oc.hu/ingatlanok/LK496857?c=79cc83b7
325 	 https://www.oc.hu/ingatlanok/LK496177?c=79cc83b7
326 	 https://www.oc.hu/ingatlanok/LK502176?c=79cc83b7
327 	 https://www.oc.hu/ingatlanok/LK499843?c=79cc83b7
328 	 https://www.oc.hu/ingatlanok/LK498496?c=79cc83b7
329 	 https://www.oc.hu/ingatlanok/SL031842?c=79cc83b7
330 	 https://www.oc.hu/ingatlanok/LK478521?c=79cc83b7
331 	 https://www.oc.hu/ingatlanok/H351175?c=79cc83b7
332 	 https://www.oc.hu/ingatlanok/H361478?c=79cc83b7
333 	 https:/

49 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/42-millio-ft;50-negyzetmeter;2-szoba;tegla-epitesu/9790721
50 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/42-millio-ft;51-negyzetmeter;1-1-szoba;nincs-megadva-epitesu/9941460
51 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/38-millio-ft;48-negyzetmeter;1-1-szoba;tegla-epitesu/9703725
52 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/56-millio-ft;51-negyzetmeter;2-szoba;nincs-megadva-epitesu/9248272
53 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/55-millio-ft;54-negyzetmeter;2-szoba;tegla-epitesu/9872519
54 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/34-millio-ft;50-negyzetmeter;2-szoba;panel-epitesu/9899661
55 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/27-millio-ft;49-negyzetmeter;2-szoba;nincs-megadva-epitesu/9941393
56 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/61-millio-ft;53-negyzetmeter;3-szoba;tegla-epitesu/9967148
57 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/36-mi

124 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+andor+utca/31213351
125 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+menyecske+utca/31084642
126 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+andor+utca+11a/30987649
127 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+sopron+ut+63/31437407
128 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+etele+ut/31339399
129 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+hadak+utja+2/31439426
130 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31427553
131 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/30877660
132 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fehervari+ut/30543865
133 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fehervari+ut/30823576
134 	 https://ingatl

209 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+safrany+utca/31375084
210 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+allende+park/31366855
211 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31157335
212 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+safrany+utca/31047616
213 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+frakno+utca/30182156
214 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31378561
215 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+frakno+utca/31354066
216 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31344097
217 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+safrany+utca/31232974
218 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+safrany+utca/28950401
219 	 https://ingatlan.com/xi-ker/elado+l

292 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+tetenyi+ut+42/31172605
293 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fehervari+ut/31160020
294 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+hadak+utja+6/30967729
295 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/30634741
296 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fehervari+ut/30211382
297 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+kokorcsin+utca/29328896
298 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+tetenyi+ut/31428806
299 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+somogyi+ut/30289424
300 	 https://www.oc.hu/ingatlanok/H406630?c=434f5352
301 	 https://www.oc.hu/ingatlanok/H392169?c=434f5352
302 	 https://www.oc.hu/ingatlanok/H397657?c=434f5352
303 	 https://www.oc.hu/ingatlanok/H

In [13]:
import os
for i in sorted(os.listdir()):
    if 'ads' in i:
        print(i,round(os.stat(i).st_size/(1024*1024),2), 'M')

adscenVI_03032020.csv 5.18 M
adscenVI_05042020.csv 5.25 M
adscenVI_08032020.csv 5.19 M
adscenVI_13042020.csv 0.0 M
adscenVI_15032020.csv 5.23 M
adscenVI_22032020.csv 5.24 M
adscenVI_29032020.csv 5.25 M
adscenV_01032020.csv 0.2 M
adscenV_05042020.csv 0.3 M
adscenV_08032020.csv 0.2 M
adscenV_13042020.csv 0.0 M
adscenV_15032020.csv 0.2 M
adscenV_24022020.csv 0.18 M
adscenV_29032020.csv 0.3 M
adscenXI_04032020.csv 2.96 M
adscenXI_05042020.csv 2.78 M
adscenXI_08032020.csv 3.06 M
adscenXI_13042020.csv 0.0 M
adscenXI_15032020.csv 2.99 M
adscenXI_22032020.csv 2.9 M
adscenXI_29032020.csv 2.78 M
adscomVI_03032020.csv 17.44 M
adscomVI_05042020.csv 17.77 M
adscomVI_08032020.csv 17.5 M
adscomVI_13042020.csv 0.0 M
adscomVI_15032020.csv 17.52 M
adscomVI_22032020.csv 17.41 M
adscomVI_29032020.csv 17.46 M
adscomV_01032020.csv 4.04 M
adscomV_05042020.csv 4.65 M
adscomV_08032020.csv 3.86 M
adscomV_13042020.csv 0.0 M
adscomV_15032020.csv 4.14 M
adscomV_22032020.csv 4.38 M
adscomV_24022020.csv 4.11 M
adsco

In [14]:
checkDownload('30032020', ['XI'])

Saved ads in adshuXI: 100
Saved ads in adscomXI: 200
Saved ads in adscenXI: 25


In [14]:
ads

[([], [], []), ([], [], []), ([], [], [])]